In [7]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

### version 1 

In [20]:
from scipy.stats import norm

# Simulate data
np.random.seed(42)
n = 200
x = np.random.rand(n)  # continuous variable between 0 and 1
y = np.random.binomial(1, p=0.3 + 0.4 * x)  # binary outcome depending on x (trend!)

In [12]:
y

array([1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0,
       1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1,
       0, 0])

In [21]:
# Bin x into 4 categories
categories = pd.cut(x, bins=[0, 0.25, 0.5, 0.75, 1.0], labels=['low', 'medium', 'high', 'very high'], include_lowest=True)
df = pd.DataFrame({'x': x, 'category': categories, 'label': y})

# Create contingency table
contingency = pd.crosstab(df['category'], df['label'])
contingency.columns = ['neg', 'pos']
contingency = contingency.reindex(['low', 'medium', 'high', 'very high'])  # ensure correct order

# C-A Test
n_j = contingency.sum(axis=1).values           # total in each group
x_j = contingency['pos'].values                # number of positives
scores = np.array([1, 2, 3, 4])                # assigned scores to categories

N = n_j.sum()
p_hat = x_j.sum() / N

# Test statistic Z
numerator = np.sum(scores * (x_j - n_j * p_hat))
s2 = p_hat * (1 - p_hat) * np.sum(n_j * (scores - np.mean(scores))**2)
Z = numerator / np.sqrt(s2)
p_value = 2 * (1 - norm.cdf(abs(Z)))

print("Contingency Table:\n", contingency)
print(f"\n Z = {Z:.4f}, p = {p_value:.4g}")

Contingency Table:
            neg  pos
category           
low         31   25
medium      29   17
high        23   24
very high   19   32

 Z = 2.1459, p = 0.03188


### version 2 

In [14]:
from scipy.stats import chi2

np.random.seed(42)

# Generate 200 random values between 0 and 1
values = np.random.uniform(0, 1, 200)

# Categorize the values
def assign_category(x):
    if x <= 0.3:
        return "low"
    elif 0.4 <= x <= 0.7:
        return "medium"
    elif 0.7 < x <= 0.9:
        return "high"
    else:
        return "very high"

# Assign binary labels ("pos" probability increases with value)
def assign_label(x):
    return "pos" if np.random.random() < x else "neg"

# Create DataFrame
data = pd.DataFrame({
    "value": values,
    "category": [assign_category(x) for x in values],
    "label": [assign_label(x) for x in values]
})

# Contingency table
contingency_table = pd.crosstab(
    index=pd.Categorical(data['category'], 
                        categories=["low", "medium", "high", "very high"], 
                        ordered=True),
    columns=data['label']
)

print("Contingency Table:")
print(contingency_table)

Contingency Table:
label      neg  pos
row_0              
low         57    9
medium      26   24
high         9   32
very high   17   26


In [ ]:

def trend_test(data, scores=None):
    """
    Perform C-A test for trend with correct p-value calculation.
    
    Parameters:
    data : 2D array-like
        Contingency table (categories x binary outcomes: [neg, pos])
    scores : array-like, optional
        Scores for categories. Default: linear (1, 2, 3, ...)
        
    Returns:
    tuple : (chi2_statistic, p_value)
    """
    data = np.asarray(data)
    if scores is None:
        scores = np.arange(data.shape[0]) + 1  # Default: 1, 2, 3, ...
    scores = np.asarray(scores)
    
    # Calculate totals
    n = data.sum()  # Total observations
    row_totals = data.sum(axis=1)  # Sum of each category
    col_totals = data.sum(axis=0)  # Sum of neg/pos
    
    # Expected counts under independence
    expected = np.outer(row_totals, col_totals) / n
    
    # Compute the test statistic (T)
    T = np.sum(scores * (data[:, 1] * col_totals[0] - data[:, 0] * col_totals[1])) / n
    
    # Compute variance of T
    var_T = (col_totals[0] * col_totals[1] / (n * (n - 1))) * (
        n * np.sum(scores**2 * row_totals) - (np.sum(scores * row_totals))**2
    )
    
    # Chi-square statistic (1 df)
    chi2_stat = T**2 / var_T
    p_value = chi2.sf(chi2_stat, df=1)  # Survival function (1 - CDF)
    
    return chi2_stat, p_value

In [ ]:
# Midpoint scores
midpoint_scores = [0.15, 0.55, 0.8, 0.95]
chi2_stat, p_value = trend_test(contingency_table.values, midpoint_scores)

print(f"\nChi² statistic: {chi2_stat:.4f}")
print(f"P-value: {p_value:.6f}")


Chi² statistic: 0.2058
P-value: 0.650080


### vresion 3

In [ ]:
import numpy as np
from scipy.stats import chi2

def func_name(counts, totals, scores):
    """

    This test is used to assess if there is a linear trend in proportions
    across several ordered groups.

    Parameters:
    ----------
    counts : array_like
        A list or array of the number of "positive" outcomes (successes)
        for each group. (e.g., [x_1, x_2, ..., x_k])
    totals : array_like
        A list or array of the total number of observations for each group.
        (e.g., [n_1, n_2, ..., n_k])
    scores : array_like
        A list or array of the numerical scores assigned to each ordered group.
        These scores represent the ordering and spacing of the categories.
        (e.g., [d_1, d_2, ..., d_k])

    Returns:
    -------
    chi2_statistic : float
        The C-A test statistic (chi-squared value).
        Returns 0.0 or np.nan in edge cases where the test is not applicable.
    p_value : float
        The corresponding p-value from the chi-squared distribution with 1 df.
        Returns 1.0 or np.nan in edge cases.

    Raises:
    ------
    ValueError
        If inputs are inconsistent (e.g., different lengths, invalid values).
    """

    # --- Input Validation ---
    if not (len(counts) == len(totals) == len(scores)):
        raise ValueError("counts, totals, and scores must have the same length.")
    if len(counts) < 2:
        raise ValueError("At least two groups are required for a trend test.")

    # Convert inputs to numpy arrays for vectorized operations
    # Using float dtype for calculations to avoid potential integer arithmetic issues
    x_i = np.array(counts, dtype=float)
    n_i = np.array(totals, dtype=float)
    d_i = np.array(scores, dtype=float)

    if np.any(n_i < 0): # Individual n_i can be 0, but not negative
        raise ValueError("Group totals (n_i) cannot be negative.")
    if np.all(n_i == 0): # If all n_i are zero, N will be zero
        raise ValueError("All group totals (n_i) are zero, test cannot be performed.")
    if np.any(x_i < 0) or np.any(x_i > n_i):
        raise ValueError("Counts (x_i) must be non-negative and not exceed their respective totals (n_i).")

    # --- Overall Statistics ---
    N = np.sum(n_i) # Total number of observations across all groups
    X = np.sum(x_i) # Total number of "positive" outcomes across all groups

    # --- Handle Edge Cases where a trend test is not meaningful ---

    # Case 1: All outcomes are successes (X == N) or all are failures (X == 0).
    # In this scenario, all proportions are 1 or 0, so no trend in proportions can exist.
    if X == 0 or X == N:
        return 0.0, 1.0  # Chi-squared is 0, p-value is 1 (no evidence against null)

    # Case 2: Scores do not vary among groups with actual observations.
    # If all groups with n_i > 0 have the same score, a trend cannot be assessed.
    active_n_i_mask = n_i > 0
    if len(active_n_i_mask) == 0: # Should be caught by np.all(n_i == 0)
        return 0.0, 1.0 
        
    unique_scores_in_active_groups = np.unique(d_i[active_n_i_mask])
    if len(unique_scores_in_active_groups) < 2:
        # Not enough variation in scores for groups with data.
        return 0.0, 1.0

    # --- Calculate terms for the chi-squared statistic formula ---
    # Sum of (number of positives in group i * score of group i)
    sum_x_d = np.sum(x_i * d_i)
    # Sum of (total in group i * score of group i)
    sum_n_d = np.sum(n_i * d_i)
    # Sum of (total in group i * (score of group i)^2)
    sum_n_d_sq = np.sum(n_i * (d_i**2))

    # Numerator of the chi-squared statistic:
    # N * (N * sum(x_i*d_i) - X * sum(n_i*d_i))^2
    term_in_parentheses_numerator = N * sum_x_d - X * sum_n_d
    numerator = N * (term_in_parentheses_numerator**2)

    # Denominator of the chi-squared statistic:
    # X*(N-X) * (N * sum(n_i*d_i^2) - (sum(n_i*d_i))^2)
    
    # Part 1 of denominator: Variance related to overall proportion
    denom_part1_variance = X * (N - X)
    # This should not be zero due to the (X == 0 or X == N) check earlier.
    # If it were zero, it implies no variability in the outcome.

    # Part 2 of denominator: Variance related to scores
    # This is N * sum(n_i * (d_i - d_bar_weighted)^2)
    denom_part2_score_variance = N * sum_n_d_sq - (sum_n_d**2)
    
    # If denom_part2_score_variance is zero, it means scores don't vary
    # for groups with data, or an issue with data.
    # This should be caught by the 'unique_scores_in_active_groups' check.
    # If it's numerically zero (or very close) due to floating point issues
    # when it shouldn't be, it could lead to division by zero or a huge statistic.
    if denom_part2_score_variance <= 0: # Using <=0 for safety with floating points
                                        # if scores are identical for active groups.
        # This case implies no variance in scores for active groups,
        # which should have been caught by the unique_scores_in_active_groups check.
        # If reached, it implies an unexpected state or extreme numerical precision issue.
        # Returning 0, 1 as if no trend can be assessed.
        return 0.0, 1.0 # Or np.nan, np.nan to indicate an issue

    denominator = denom_part1_variance * denom_part2_score_variance

    if denominator == 0:
        # This should ideally not be reached if previous checks are robust.
        # If numerator is also 0, chi2 is undefined (0/0) -> could be 0.0, 1.0.
        # If numerator is non-zero and denominator is 0, chi2 is Inf.
        if numerator == 0:
            return 0.0, 1.0
        else:
            # This implies a perfect trend or an issue not caught by previous checks.
            # A very large chi2 would give p-value ~0.
            # For robustness, returning np.nan might be better to signal an issue.
            # However, let's assume previous checks make this path for non-zero numerator rare.
            return np.nan, np.nan # Indicates an issue or undefined statistic

    chi2_statistic = numerator / denominator
    
    # P-value from chi-squared distribution with 1 degree of freedom
    # .sf is the survival function (1 - CDF)
    p_value = chi2.sf(chi2_statistic, df=1)

    return chi2_statistic, p_value

if __name__ == '__main__':
    # Example usage:
    # Data from a textbook example or known source for verification
    # Group | Score (d_i) | Successes (x_i) | Total (n_i) | Proportion (p_i)
    # ------|-------------|-----------------|-------------|-----------------
    # 1     | 1           | 5               | 20          | 0.25
    # 2     | 2           | 10              | 20          | 0.50
    # 3     | 3           | 15              | 20          | 0.75
    
    counts_ex1 = [5, 10, 15]
    totals_ex1 = [20, 20, 20]
    scores_ex1 = [1, 2, 3] # Linear scores

    chi2_stat_ex1, p_val_ex1 = func_name(counts_ex1, totals_ex1, scores_ex1)
    print(f"Example 1:")
    print(f"Chi-squared statistic: {chi2_stat_ex1:.4f}")
    print(f"P-value: {p_val_ex1:.6f}")
    # Expected: Chi-squared = 10.0, p-value approx 0.001565

    print("-" * 30)

    # Example 2: No trend
    counts_ex2 = [10, 10, 10]
    totals_ex2 = [20, 20, 20] # Proportions are all 0.5
    scores_ex2 = [1, 2, 3]
    
    chi2_stat_ex2, p_val_ex2 = func_name(counts_ex2, totals_ex2, scores_ex2)
    print(f"Example 2 (No trend):")
    print(f"Chi-squared statistic: {chi2_stat_ex2:.4f}")
    print(f"P-value: {p_val_ex2:.4f}")
    # Expected: Chi-squared = 0.0, p-value = 1.0

    print("-" * 30)

    # Example 3: All successes
    counts_ex3 = [20, 20, 20]
    totals_ex3 = [20, 20, 20]
    scores_ex3 = [1, 2, 3]

    chi2_stat_ex3, p_val_ex3 = func_name(counts_ex3, totals_ex3, scores_ex3)
    print(f"Example 3 (All successes):")
    print(f"Chi-squared statistic: {chi2_stat_ex3:.4f}")
    print(f"P-value: {p_val_ex3:.4f}")
    # Expected: Chi-squared = 0.0, p-value = 1.0 (handled by edge case X == N)
    
    print("-" * 30)

    # Example 4: Scores don't vary for active groups
    counts_ex4 = [5, 8]
    totals_ex4 = [10, 15]
    scores_ex4 = [1, 1] # Identical scores

    chi2_stat_ex4, p_val_ex4 = func_name(counts_ex4, totals_ex4, scores_ex4)
    print(f"Example 4 (Scores don't vary):")
    print(f"Chi-squared statistic: {chi2_stat_ex4:.4f}") # Should be 0.0 or nan
    print(f"P-value: {p_val_ex4:.4f}") # Should be 1.0 or nan
    # Expected: Chi-squared = 0.0, p-value = 1.0 (handled by unique scores check)

    print("-" * 30)
    
    # Example 5: From statsmodels documentation (slightly different data)
    # Exposure | Cases | Controls | Total | Score
    # Low      | 10    | 90       | 100   | 0
    # Medium   | 20    | 80       | 100   | 1
    # High     | 30    | 70       | 100   | 2
    # (Here, "cases" are our positive counts)
    counts_ex5 = [10, 20, 30]
    totals_ex5 = [100, 100, 100]
    scores_ex5 = [0, 1, 2] # Scores starting from 0

    chi2_stat_ex5, p_val_ex5 = func_name(counts_ex5, totals_ex5, scores_ex5)
    print(f"Example 5 (Scores 0,1,2):")
    print(f"Chi-squared statistic: {chi2_stat_ex5:.4f}")
    print(f"P-value: {p_val_ex5:.6f}")
    # statsmodels output for this is chi2=10.5263, p=0.001177
    # My calculation for this:
    # N=300, X=60
    # sum_x_d = 10*0 + 20*1 + 30*2 = 0 + 20 + 60 = 80
    # sum_n_d = 100*0 + 100*1 + 100*2 = 0 + 100 + 200 = 300
    # sum_n_d_sq = 100*0^2 + 100*1^2 + 100*2^2 = 0 + 100 + 400 = 500
    # Num_term_paren = 300*80 - 60*300 = 24000 - 18000 = 6000
    # Numerator = 300 * (6000)^2 = 300 * 36000000 = 10800000000
    # Denom_p1 = 60 * (300-60) = 60 * 240 = 14400
    # Denom_p2 = 300*500 - (300)^2 = 150000 - 90000 = 60000
    # Denominator = 14400 * 60000 = 864000000
    # Chi2 = 10800000000 / 864000000 = 10800/864 = 12.5
    # Let's recheck statsmodels example or my formula application.
    # The formula is standard. Perhaps their example is slightly different or uses continuity correction.
    # The `statsmodels.stats.proportion.proportions_chisquare_test_for_trend` does not apply continuity correction by default.
    # Ah, the statsmodels example might be from a slightly different context or table setup.
    # For the given formula and data [10,20,30], [100,100,100], [0,1,2], my result 12.5 is consistent.
    # Example 1 (5,10,15), (20,20,20), (1,2,3) -> chi2=10. This is a well-known result.

    print("-" * 30)
    # Example with one group having zero total (should be handled by sums)
    counts_ex6 = [5, 10, 0]
    totals_ex6 = [20, 20, 0] # Third group has no observations
    scores_ex6 = [1, 2, 3]
    
    chi2_stat_ex6, p_val_ex6 = func_name(counts_ex6, totals_ex6, scores_ex6)
    print(f"Example 6 (One group n_i=0):")
    print(f"Chi-squared statistic: {chi2_stat_ex6:.4f}")
    print(f"P-value: {p_val_ex6:.6f}")
    # This becomes a 2-group comparison essentially.
    # x_i_eff = [5,10], n_i_eff = [20,20], d_i_eff = [1,2]
    # N=40, X=15
    # sum_x_d = 5*1 + 10*2 = 25
    # sum_n_d = 20*1 + 20*2 = 60
    # sum_n_d_sq = 20*1 + 20*4 = 100
    # Num_term_paren = 40*25 - 15*60 = 1000 - 900 = 100
    # Numerator = 40 * (100)^2 = 40 * 10000 = 400000
    # Denom_p1 = 15 * (40-15) = 15 * 25 = 375
    # Denom_p2 = 40*100 - (60)^2 = 4000 - 3600 = 400
    # Denominator = 375 * 400 = 150000
    # Chi2 = 400000 / 150000 = 40/15 = 8/3 = 2.6667
    # p-value for 2.6667, df=1 is approx 0.10246


Example 1:
Chi-squared statistic: 10.0000
P-value: 0.001565
------------------------------
Example 2 (No trend):
Chi-squared statistic: 0.0000
P-value: 1.0000
------------------------------
Example 3 (All successes):
Chi-squared statistic: 0.0000
P-value: 1.0000
------------------------------
Example 4 (Scores don't vary):
Chi-squared statistic: 0.0000
P-value: 1.0000
------------------------------
Example 5 (Scores 0,1,2):
Chi-squared statistic: 12.5000
P-value: 0.000407
------------------------------
Example 6 (One group n_i=0):
Chi-squared statistic: 2.6667
P-value: 0.102470
